In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *

gtimer = GlobalTimer.instance()
gtimer.reset()

In [2]:
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *
crob = CombinedRobot(connection_list=(False, False))

connection_list
(False, False)


## Global params definition

In [3]:
GLOBAL_PARAMS = dict(
    BASE_LINK = "base_link",
    MAX_REACH_DICT = {'indy0': 1.5, 'panda1': 1.5},
    REACH_OFFSET_DICT = {'indy0': (0,0,0.3), 'panda1': (0,0,0.3)},
    GRIPPER_REFS = {"indy0":{"link_name": "indy0_tcp", "tcp_ref": [0,0,0.14], "depth_range": (-0.07,0.00), "width_range": (0.03,0.05)}, 
                    "panda1":{"link_name": "panda1_hand", "tcp_ref": [0,0,0.112], "depth_range": (-0.04,0.00), "width_range": (0.03,0.06)}
                   },
    Nrbt = len(crob.robot_names), # 2
    Njoints = len(crob.joint_names), # 13
    WDH = (3,3,3),
    WDH_MIN_RBT = (0.75,0.75,0.5), 
    WDH_MAX_RBT = (2.25,2.25,1.0),
    L_CELL = 0.2,
    MIN_DIST_ROBOT = 1,
    Ns = 100,
    RATIO_COVER = 2.0,
    RATIO_DIMS = 2.0,
    LINK_COLL_MARGIN = 0.01
)
SAMPLED_DATA = defaultdict(dict)

## set params

In [4]:
__local_params = locals()
for k,v in GLOBAL_PARAMS.items():
    __local_params[k] = v
CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
Ws, Ds, Hs = WDH 
Nw, Nd, Nh = Nwdh = int(Ws/L_CELL), int(Ds/L_CELL), int(Hs/L_CELL)
L_MAX = L_CELL*RATIO_DIMS
print("scene size: {} ({},{},{})".format(Nw*Nd*Nh, Nw, Nd, Nh))

scene size: 3375 (15,15,15)


In [5]:
SAMPLED_DATA["WORLD"]["Trbt_dict"] = Trbt_dict= sample_Trbt(Nrbt, crob.robot_names, WDH_MIN_RBT, WDH_MAX_RBT, MIN_DIST_ROBOT)
reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

# LEVEL: WORLD

## put robot

In [6]:
cam = None

# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict, crob.joint_names)
time.sleep(2)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


In [7]:
# set graph
if "graph" in locals():
    graph.clear_markers()
    graph.clear_highlight()
    graph.ghnd.clear()
    graph.__init__(urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                   urdf_content=urdf_content, combined_robot=crob)
    graph.set_cam_robot_collision(_add_cam_poles=False)
    graph.set_rviz()
else:
    graph = ConstraintGraph(urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                            urdf_content=urdf_content, combined_robot=crob)
    graph.set_camera(cam)
    graph.set_cam_robot_collision(_add_cam_poles=False)
    graph.set_rviz()
    
    # start UI
    ui_broker = UIBroker(graph)
    ui_broker.start_server()
    
    # set rviz
    graph.set_rviz(crob.home_pose)
    ui_broker.set_tables()

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## show workspace

In [8]:
# show_workspace(graph, Nwdh, CENTER, L_CELL)

In [9]:
# remove_geometries_by_prefix(graph, "workspace")
# remove_geometries_by_prefix(graph, "grid")

# LEVEL: PREMISE

## init joints

In [10]:
from pkg.data_collecting.sampling import *

In [11]:
SAMPLED_DATA["PREMISE"]["Q_s"], _, _, _, _ = Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph)
SAMPLED_DATA["PREMISE"]["Q_s_dict"] = Q_s_dict = list2dict(Q_s, crob.joint_names)
        
graph.show_pose(Q_s)
time.sleep(1)
graph.show_pose(Q_s)
time.sleep(1)
graph.show_pose(Q_s)

## extract available cells

In [12]:
gtimer.tic("extract_available")
# coll_boxes = get_colliding_cells(Nwdh, L_CELL, link_ctems, link_rads)
free_boxes = get_reachable_cells(Nwdh, L_CELL, reach_center_dict, MAX_REACH_DICT)
gtimer.toc("extract_available")

# draw_cells(graph, "col_cell", coll_boxes, L_CELL, color=(0.9, 0, 0.2, 0.1))
# remove_geometries_by_prefix(graph, "col_cell")
# draw_cells(graph, "reachable", free_boxes, L_CELL, color=(0.2, 0.7, 0.2, 0.1))
# remove_geometries_by_prefix(graph, "reachable")

2.7709007263183594

# LEVEL: SCENE

## sample objects

In [13]:
obj_dat = []
for geo_gen in OBJ_GEN_LIST:
    obj_boxes = random.sample(free_boxes,Ns)
#     draw_cells(graph, "obj_cell", obj_boxes, L_CELL, color=(0.2, 0.2, 0.7, 0.1))
#     remove_geometries_by_prefix(graph, "obj_cell")
    for nbox in obj_boxes:    
        gtype, dims, color = geo_gen(L_MAX)
        obj_dat.append({"nbox": nbox, "gtype": gtype.name, "dims": dims, "color": color, 
                        "center": tuple(np.multiply(nbox, L_CELL)+np.random.random((3,))*L_MAX+0.5*L_CELL - 0.5*L_MAX), 
                        "rpy": np.random.random((3,))*np.pi*2})
SAMPLED_DATA["SCENE"]["obj_dat"] = obj_dat

## exclude colliding object

In [14]:
__obj_dat=[]
coll_list = []
for odat in obj_dat:
    gtype, center, rpy, dims = getattr(GEOTYPE,odat['gtype']), odat['center'], odat['rpy'], odat['dims']
    ctem = getPointList(get_vertex_rows(gtype, center, rpy, dims))
    crad = (dims[0]/2) if gtype in [GEOTYPE.CAPSULE, GEOTYPE.CYLINDER, GEOTYPE.SPHERE] else 0
    if any(np.array(get_distance_list(ctem, link_ctems, crad, link_rads))<LINK_COLL_MARGIN):
        odat['color'] = (0.9, 0.0, 0.0, 0.3)
        coll_list.append(odat)
    else:
        __obj_dat.append(odat)
SAMPLED_DATA["SCENE"]["obj_dat"] = obj_dat = __obj_dat

## put objects

In [15]:
if "obj_list" in locals(): 
    for obj in obj_list: graph.remove_geometry(obj)
obj_list=[]
for odat in obj_dat:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="{}_{}_{}_{}".format(gtype.name,*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
    obj_list.append(obj)
    graph.add_geometry(obj)

In [16]:
if "col_obj_list" in locals(): 
    for obj in col_obj_list: graph.remove_geometry(obj)
col_obj_list=[]
for odat in coll_list:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="col_obj_{}_{}_{}".format(*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=False, fixed=True)
    col_obj_list.append(obj)
    graph.add_geometry(obj)

In [17]:
if "col_obj_list" in locals(): 
    for obj in col_obj_list: graph.remove_geometry(obj)

# LEVEL: ACTION

## pick case

In [18]:
#select gripper
rname, gripper = random.choice(GRIPPER_REFS.items())
depth_range = gripper['depth_range']
width_range = gripper['width_range']
depth_abs = -depth_range[0]

In [19]:
obj = random.choice(obj_list)
color_bak = obj.color
obj.color = (1,0,0,0.5)
graph.add_geometry(obj)

In [20]:
T_ygrip, dims_new, dims_bak = sample_grasp(
    obj, WIDTH_RANGE=width_range, DEPTH_RANGE=depth_range, DIM_MAX=L_MAX, fit_dim=True)
obj.dims = dims_new
T_ygrip_link = np.matmul(obj.Toff, T_ygrip)
graph.remove_geometry(obj)
graph.add_geometry(obj)
graph.add_highlight_axis(obj.name, "obj_axis", obj.link_name, obj.Toff[:3,3], obj.Toff[:3,:3], color=None, axis="xyz", dims=(0.03, 0.003, 0.003))
graph.add_highlight_axis(obj.name, "grip_axis", obj.link_name, T_ygrip_link[:3,3], 
                         T_ygrip_link[:3,:3], color=None, axis="xyz", dims=(depth_abs, depth_abs/10, depth_abs/10))
#                          np.matmul(T_ygrip_link[:3,:3], Rot_axis(2,np.pi)), color=None, axis="xyz", dims=(depth_abs, depth_abs/10, depth_abs/10))

In [21]:
T_lg = SE3(np.identity(3), gripper['tcp_ref'])
T_lo = np.matmul(T_lg, SE3_inv(T_ygrip))
inhand = graph.ghnd.create_safe(
    name="inhand", link_name=gripper["link_name"], gtype=obj.gtype, 
    center=T_lo[:3,3], rpy=Rot2rpy(T_lo[:3,:3]), dims=obj.dims, 
    color=(1,0,0,0.5), display=True, collision=False, fixed=False)
graph.remove_geometry(inhand)
graph.add_geometry(inhand)
graph.add_highlight_axis("inhand", "inhand_axis", gripper["link_name"], gripper["tcp_ref"], np.identity(3), color=None, axis="xyz", dims=(0.03, 0.003, 0.003))

In [22]:
SAMPLED_DATA["ACTION"]["robot"] = rname
SAMPLED_DATA["ACTION"]["target_obj"] = obj.name
SAMPLED_DATA["ACTION"]["T_ygrip"] = T_ygrip
SAMPLED_DATA["ACTION"]["obj_dims"] = dims_new
SAMPLED_DATA["ACTION"]["obj_dims_bak"] = dims_bak
SAMPLED_DATA["ACTION"]["color_bak"] = color_bak

## recover object dimension

In [23]:
graph.clear_highlight("inhand")
graph.clear_highlight(obj.name)
graph.remove_geometry(inhand)
obj.dims, obj.color = dims_bak, color_bak
graph.remove_geometry(obj)
graph.add_geometry(obj)

## place case

In [24]:
#select gripper
rname, gripper = random.choice(GRIPPER_REFS.items())
depth_range = gripper['depth_range']
width_range = gripper['width_range']
depth_abs = -depth_range[0]

In [25]:
tar = random.choice(obj_list)
T_lp, T_zplace = sample_putpoint(tar)
graph.add_highlight_axis(tar.name, "pp_axis", tar.link_name, T_lp[:3, 3], T_lp[:3, :3], color=None, axis="xyz",
                         dims=(0.05, 0.005, 0.005))

In [26]:
ontarget, T_lo = sample_putobject(graph, tar, T_lp, L_CELL)
graph.remove_geometry(ontarget)
graph.add_geometry(ontarget)
graph.add_highlight_axis(tar.name, "obj_axis", tar.link_name, T_lo[:3,3], T_lo[:3,:3], color=None, axis="xyz", dims=(0.05, 0.005, 0.005))

In [27]:
T_ygrip, dims_new, dims_bak = sample_grasp(
    ontarget, WIDTH_RANGE=width_range, DEPTH_RANGE=depth_range, DIM_MAX=L_MAX, fit_dim=False)
T_ygrip_link = np.matmul(ontarget.Toff, T_ygrip)
graph.add_highlight_axis(tar.name, "ontarget_axis", tar.link_name, T_ygrip_link[:3,3], 
                         T_ygrip_link[:3,:3], color=None, axis="xyz", dims=(depth_abs, depth_abs/10, depth_abs/10))

In [28]:
T_glo = np.matmul(SE3(np.identity(3),gripper['tcp_ref']), SE3_inv(T_ygrip))
inhand = graph.ghnd.create_safe(
    name="inhand", link_name=gripper["link_name"], gtype=ontarget.gtype, 
    center=T_glo[:3,3], rpy=Rot2rpy(T_glo[:3,:3]), dims=ontarget.dims, 
    color=(1,0,0,0.5), display=True, collision=True, fixed=False)
graph.remove_geometry(inhand)
graph.add_geometry(inhand)

In [29]:
graph.add_highlight_axis(tar.name, "gripper_axis", gripper['link_name'], gripper['tcp_ref'], 
                         np.identity(3), color=None, axis="xyz", dims=(depth_abs, depth_abs/10, depth_abs/10))

In [30]:
graph.clear_highlight(tar.name)
graph.remove_geometry(inhand)
graph.remove_geometry(ontarget)

## handover case - make in-hand object

In [39]:
src, tar = random.sample(GRIPPER_REFS.items(),2)

In [40]:
gtype, dims, color = random.choice(OBJ_GEN_LIST)(L_CELL)

In [41]:
handed = graph.ghnd.create_safe(gtype=gtype, name="handed_in_src", link_name=src[1]['link_name'], 
                                center=(0,0,0), rpy=(0,0,0), dims=dims, color=(0,1,1,0.5), 
                                display=True, collision=True, fixed=False)

In [42]:
Ttar_ygrip, dims_new, dims_bak = sample_grasp(
    handed, WIDTH_RANGE=tar[1]['width_range'], DEPTH_RANGE=tar[1]['depth_range'], 
    DIM_MAX=L_MAX, fit_dim=True)
handed.dims = dims_new

In [43]:
Tsrc_ygrip, dims_new, dims_bak = sample_grasp(
    handed, WIDTH_RANGE=src[1]['width_range'], DEPTH_RANGE=src[1]['depth_range'], 
    DIM_MAX=L_MAX, fit_dim=False)
T_slo = np.matmul(SE3(np.identity(3),src[1]['tcp_ref']), SE3_inv(Tsrc_ygrip))
handed.set_offset_tf(center=T_slo[:3,3], orientation_mat=T_slo[:3,:3])
graph.remove_geometry(handed)
graph.add_geometry(handed)

In [44]:
T_tlo = np.matmul(SE3(np.identity(3),tar[1]['tcp_ref']), SE3_inv(Ttar_ygrip))
intar = graph.ghnd.create_safe(gtype=handed.gtype, name="handed_in_tar", link_name=tar[1]['link_name'], 
                               center=T_tlo[:3,3], rpy=Rot2rpy(T_tlo[:3,:3]), dims=handed.dims, color=(1,0,0.5,0.5), 
                               display=True, collision=False, fixed=False)
graph.remove_geometry(intar)
graph.add_geometry(intar)

In [38]:
graph.remove_geometry(handed)
graph.remove_geometry(intar)